<a href="https://colab.research.google.com/github/yxyfer/EPTIA_NLP3/blob/main/lab02/SCIA_NLP3_Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import pandas as pd 

from IPython.display import HTML

In [ ]:
%%capture
! pip install fasttext

# I. Language detection (24 points)

## Setup 

In [ ]:
! git clone https://github.com/MastafaF/LanguageDetection.git

Cloning into 'LanguageDetection'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 2), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [ ]:
os.listdir("./LanguageDetection")

['README.md', 'dataset.csv.zip', '.gitignore', 'LICENSE', '.git']

In [ ]:
# CD the LanguageDetection folder - we are working in the below folder now
os.chdir("./LanguageDetection")

In [ ]:
! unzip dataset.csv.zip

Archive:  dataset.csv.zip
  inflating: dataset.csv             


## Data Exploration Analysis

In [ ]:
data = pd.read_csv("./dataset.csv")

In [ ]:
# Sample of the data
HTML(data[data.language == "Chinese"].sample().to_html())

,Text,language
16252,年月日，斯大林参加雅尔塔会议，根据协议，苏联成为联合国常任理事国成员，并将波兰等地划入自己的势力范围。月，苏联红军攻克德国首都柏林，希特勒自杀身亡，纳粹德国宣告无条件投降。同时苏军进入并控制东南欧等地区。月日，斯大林出席波茨坦会议。月日，苏联撕毁《苏日中立条约》向日本宣战，同时立即发起八月风暴行动入侵满洲。一周之内，苏蒙联军迅速击溃日本关东军和满洲国军，推翻日本扶持的傀儡政权满洲国和蒙疆自治政府。月日，裕仁天皇宣布日本投降。第二次世界大战以同盟国的胜利告终，而苏联也在战争中付出重大牺牲（死亡人数达到万人）。同年，斯大林获苏联大元帅军衔。,Chinese


### Question 1. Describe the distribution of languages and give at least two comments about the dataset. (1 point)

In [ ]:
################################################
# your implementation goes here

pass 
##################################################

### Question 2-5. Explore the data with your own preprocessing and train your model

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X=data['Text']
y=data['language']


################################################
# your implementation goes here

pass
################################################

## FastText for language detection

## FastText training setup 

In [ ]:
! wget http://downloads.tatoeba.org/exports/sentences.tar.bz2

--2021-11-01 14:44:47--  http://downloads.tatoeba.org/exports/sentences.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.tatoeba.org/exports/sentences.tar.bz2 [following]
--2021-11-01 14:44:47--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155856043 (149M) [application/octet-stream]
Saving to: ‘sentences.tar.bz2’

sentences.tar.bz2   100%[===================>] 148.64M  28.4MB/s    in 5.9s    

2021-11-01 14:44:54 (25.4 MB/s) - ‘sentences.tar.bz2’ saved [155856043/155856043]



In [ ]:
! bunzip2 sentences.tar.bz2

In [ ]:
! ls

dataset.csv  dataset.csv.zip  LICENSE  README.md  sentences.tar


In [ ]:
! tar xvf sentences.tar

sentences.csv


In [ ]:
! ls

dataset.csv  dataset.csv.zip  LICENSE  README.md  sentences.csv  sentences.tar


In [ ]:
! head -10 sentences.csv

1	cmn	我們試試看！
2	cmn	我该去睡觉了。
3	cmn	你在干什麼啊？
4	cmn	這是什麼啊？
5	cmn	今天是６月１８号，也是Muiriel的生日！
6	cmn	生日快乐，Muiriel！
7	cmn	Muiriel现在20岁了。
8	cmn	密码是"Muiriel"。
9	cmn	我很快就會回來。
10	cmn	我不知道。


In [ ]:
! awk -F"\t" '{print"__label__"$2" "$3}' < sentences.csv | shuf > all.txt

In [ ]:
! ls

all.txt      dataset.csv.zip  README.md      sentences.tar
dataset.csv  LICENSE	      sentences.csv


In [ ]:
! head -5 all.txt

__label__ber Ur bɣint ara ad d-aɣent asegzawal tafṛansit-takatalanit.
__label__epo Tom deziras riĉiĝi kaj famiĝi.
__label__deu Tom sagte, das sei bei weitem nicht gut genug.
__label__kab Amek i ilaq ara ad yekteb talɣa s tendunizit?
__label__epo Ĉu ni havas raspitan fromaĝon?


In [ ]:
! head -n 10000 all.txt > valid.txt

In [ ]:
! tail -n +10001 all.txt > train.txt

In [ ]:
! head -5 train.txt

__label__asm তাৰ পইচাৰ দৰকাৰ হৈছে।
__label__tur O, bir dolap çeviriyor.
__label__tur Bazı genç kadınlar kötü oğlanları cezbediyor.
__label__por O Japão possui muitas atrações turísticas.
__label__ukr Я довідалася про Ваш портативний копір на Tokyo Office Expo 97.


### Question 6.1. Train fasttext model on Tatoeba (2 points)

In [ ]:
%%time
import fasttext

# Check the fasttext library and implement the training
###########################################

# your implementation goes here

pass
################################################

# @TODO: Save your model when trained 
# model.save_model("langdetect.bin")

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 15.3 µs


In [ ]:
# Sanity check 
model.predict("I am French and I love English")

(('__label__eng',), array([1.00001001]))

### Question 6.2. Evaluate performance of fasttext model on valid.txt (1 point)

In [ ]:
# Hint: Create dataframe from valid.txt and evaluate performance 

###########################################

# your implementation goes here

pass
################################################

### Question 7 & 8. Test your FastText model on the same dataset as question 5.

In [ ]:
###########################################

# your implementation goes here

pass
################################################

In [ ]:
###########################################

# your implementation goes here

pass
################################################